In [1]:
country = 'Germany'
city = 'Berlin'

In [2]:

import pandas as pd
import selenium

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support import expected_conditions as EC

from housing_functions import get_driver

import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pprint
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import demjson3
import os
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import os
from dotenv import load_dotenv



from housing_functions import random_click, get_driver, search_for_place

import json

import logging
import colorlog

import sys 


In [3]:
def get_page_listings(driver):
    pass
    
def get_logger():
    handler = colorlog.StreamHandler()
    formatter = colorlog.ColoredFormatter(
        "%(log_color)s%(asctime)s [%(levelname)s] %(message)s",
        log_colors={
            "DEBUG": "cyan",
            "INFO": "green",
            "WARNING": "yellow",
            "ERROR": "red",
            "CRITICAL": "bold_red",
        }
    )
    handler.setFormatter(formatter)
    
    logger = colorlog.getLogger(__name__)
    logger.addHandler(logging.FileHandler("logging.txt", encoding="utf-8"))
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger
    
def accept_cookies(driver):
    cookies_button = driver.find_element(By.ID,'onetrust-accept-btn-handler')
    cookies_button.click()
    
    
def get_container_listings(container):
    container.find_elements(By.CLASS_NAME,'css-1efwqj7-cardLink')
    return container


def create_directory_for_photos():
    if not os.path.exists(f'{country}_{city}_house_photos'):
        os.mkdir(f'{country}_{city}_house_photos')



def get_geo_data(driver):
    try:
        latitude, longitude, number_of_rooms = None, None, None
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        script_elements = soup.find_all('script', type='application/ld+json')
        for script in script_elements:
            if script.string:
                data = json.loads(script.string)
                
                if data.get('@type') != 'Accommodation':
                    continue
                geo_data = data.get('geo')
                latitude = geo_data.get('latitude')
                longitude = geo_data.get('longitude')
                number_of_rooms = data.get('numberOfRooms')         
    except :
        print('Inside the geo_data function, an Exception has occured')    
        return None, None, None
    
    return latitude, longitude, number_of_rooms
    


In [4]:
def get_container(driver):
    page_container = driver.find_element(By.CLASS_NAME, 'css-wp5dsn-container')
    return page_container

def get_container_listings(container):
     rows = container.find_elements(By.CLASS_NAME,'css-1efwqj7-cardLink')
     return rows


def scroll_page(driver):
    new_height = driver.execute_script("return document.body.scrollHeight")





# def scrape_half_page(driver):
    
#     listings_data_of_this_page = []
    
#     container = get_container(driver)
#     container_listings = get_container_listings(container)
    
    
#     for listing in container_listings:
        
#         listing_data =scrape_listing(listing,driver)
#         listings_data_of_this_page.append(listing_data)
    
    
#     return listings_data_of_this_page
    
    
def get_listing_tags(driver):
    try:
        tags_container = driver.find_element(By.CLASS_NAME, 'css-q6fy6c-highlightContainer')
        if tags_container:
            container_elements = tags_container.find_elements(By.CLASS_NAME, 'css-1e5azn1-highlightItem')
            tags = []
            for tag_element in container_elements:
                tag = tag_element.text
                tags.append(tag)
            return tags
    except:
        return [None]
        
        
def scrape_listing_photos_and_create_their_file(driver, listing_id:str):
    folder = os.path.join(f'{country}_{city}_house_photos', listing_id)
    os.makedirs(folder, exist_ok=True)

    images_buttons = driver.find_elements(By.CLASS_NAME,'css-13emeri-tile-tileButton')
        
    # click on more photos so that more photos are loaded
    more_photos_button = driver.find_element(By.CLASS_NAME,'css-13emeri-tile-tileButton')
    more_photos_button.click()
    images = driver.find_elements(By.TAG_NAME, 'img')   
    
    unique_images = set(images)

    
    for i, image in enumerate(unique_images):
        size = image.size
        height = size['height']
        if height >= 45:
            image_url = image.get_attribute('src') or image.get_attribute('data-src') or image.get_attribute('data-lazy') or image.get_attribute('data-original')
            
            if image_url.startswith("http"):

                response = requests.get(image_url)
                image_to_be_written = response.content
                
                image_bytes = len(image_to_be_written)
                if image_bytes > 100 * 1000:
                    with open(os.path.join(folder, f'{listing_id}_{i}.jpg'), 'wb') as f1:
                        f1.write(image_to_be_written)
            else:
                print(f'image {i} doesnt start with http')
                print(image_url)

    
    
def create_metadata_file_in_the_listings_folder(listing_id,title,price,area,description,full_description,tags,latitude,longitude,number_of_rooms):
    try:
        print(listing_id.__class__)
        if not isinstance(listing_id,str):
            logger.error(f'Listing with id; {listing_id} is not of type; str')
            raise ValueError
        folder = os.path.join(f'{country}_{city}_house_photos', listing_id)    
        os.makedirs(folder, exist_ok=True)
        logger.info(f'Created folder or folder already exists for listing with id; {listing_id}')
        file = os.path.join(folder,'.metadata')
        with open (file, 'w', encoding='utf-8') as f1:
            meta_data = {
                'listing_id' : listing_id,
                'title' : title,
                'price' : price,
                'area' : area,
                'description' : description,
                'full_description' : full_description,
                'latitude' : latitude,
                'longitude'  : longitude,
                'number_of_rooms' : number_of_rooms
            }
            for i, tag in enumerate(tags):
                key = f'tag_{i}'
                meta_data[key] = tag
                
            json_meta_data = json.dumps(meta_data, ensure_ascii= False, indent=4)
            f1.write(json_meta_data)
            logger.info(f'Wrote meta_data file for listing with id; {listing_id}')
    except Exception :
        logger.error(f'Encountered error during the writing of the Meta Data file for listing with id; {listing_id}')
        driver.close()
        
        
        
        
        
        
def get_listing_text_attributes(driver):
    try:
        title = driver.find_element(By.CLASS_NAME,'css-1ql5bbl').text
        price = driver.find_element(By.CLASS_NAME, 'css-1bop1zx-pricingContent').text
        area = driver.find_element(By.CLASS_NAME, 'css-2ccjfp').text
        description = driver.find_element(By.CLASS_NAME, 'css-31lj5q').text
        full_description = driver.find_element(By.CLASS_NAME, 'css-1liw7jd-preWrap-breakWord').text

        return title, price, area, description, full_description
    
    except Exception:
        return (None,None,None,None,None,)
        


def get_listing_id(driver):
    current_url = driver.current_url
    if '/ut' in current_url:
        listing_id = current_url.split('/ut')[1].split('/de')[0]
    else:    
        listing_id = current_url.split('-')[-1]
    return listing_id



def scrape_listing(listing, driver, original_window):
    try:
        listing.click()
    except Exception:
        logger.error('Didn t manage to sclick on listing.')
    
    try:
        driver.switch_to.window(driver.window_handles[-1])
        listing_id = get_listing_id(driver)
        logger.info(f'Acquired listing id {listing_id}')
        
        title, price, area, description, full_description = get_listing_text_attributes(driver)
        logger.info(f'Title : {title}, price : {price}, area : {area}, description : {description}, fulldescription : {full_description}')
        
        tags = get_listing_tags(driver)
        logger.info(f'Tags : {tags}' )
        
        scrape_listing_photos_and_create_their_file(driver,listing_id)
        logger.info('Scraped photos')
        
        latitude, longitude, number_of_rooms = get_geo_data(driver)
        logger.info(f'Latitude {latitude}, longitude : {longitude}, Number of Rooms : {number_of_rooms}')

        create_metadata_file_in_the_listings_folder(listing_id,title,price,area,description,full_description,tags,latitude,longitude,number_of_rooms)
        logger.info(f'Created metadata file for listing with id ; {listing_id}')
    
    except Exception:
        logger.error('something failed when scraping the data')
    finally:
        
        if driver.current_window_handle != original_window:
            driver.close()
            driver.switch_to.window(original_window)


<button class="MuiButtonBase-root MuiButton-root MuiButton-contained MuiButton-containedPrimary MuiButton-sizeMedium MuiButton-containedSizeMedium MuiButton-fullWidth MuiButton-root MuiButton-contained MuiButton-containedPrimary MuiButton-sizeMedium MuiButton-containedSizeMedium MuiButton-fullWidth css-tkyhxc-button-button" tabindex="0" type="submit" data-test-locator="Search and book">Search<span class="MuiTouchRipple-root css-w0pj6f"></span></button>

******** PROGRAM STARTS

In [5]:
logger = get_logger()
driver = get_driver()
create_directory_for_photos()
#ipython = get_ipython()


In [6]:
tries = 0
while True:
    try:
        tries += 1
        if tries == 2:
            os.execv(sys.executable, ['python'] + sys.argv)

        search_for_place(driver)
        accept_cookies(driver)
        break

    except Exception :
        logger.error('Exception while trying to search for place')

In [ ]:
def scrape_half_page(driver):
    try:
        logger.info('Trying to get the container element')
        container = driver.find_element(By.CLASS_NAME, 'css-wp5dsn-container')
        
        scroll_page(driver)
        logger.info('Trying to get the container\'s listings')
        container_listings = get_container_listings(container)
        
        original_window = driver.current_window_handle

        logger.info('Iterating over the container\'s listings')
        for  listing in container_listings:
            try:
                logger.info('Trying to scrape a listing from the container')
                listing_data = scrape_listing(listing, driver, original_window)
            except ElementClickInterceptedException:
                logger.error('Failed to scrape a listing from the container')
    except Exception :
        logger.error('Undefined error')

    finally:
        print('containers;', len(container_listings))

def scrape_page(driver):
    '''
    For each page, scrape its upper half first.
    Then scroll down a bit, to get more container listings, and scrape the other half
    '''
    scrape_half_page(driver)
    driver.execute_script("window.scrollBy(0, 2000);")
    scrape_half_page(driver)


def go_to_next_page(driver):
    logger.info('Trying to go to the next Page')
    next_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='Go to next page']")
    next_button.click()
    logger.info('Clicked on next Page')


def scrape_pages(driver):
    logger.info('Started Scraping Pages')
    pages_scraped = 0
    try:
        while True:
            try:
                logger.info(f'Trying to Scrape the {pages_scraped}th Page')
                #scrape_page(driver)
                logger.info(f'Successfuly scraped the {pages_scraped}th Page')
                
                pages_scraped += 1        
                
                logger.info(f'Trying to click on the {pages_scraped}th Page')
                go_to_next_page(driver)
                
                if pages_scraped == 100: # we do not need to scrape more than 100 pages
                    logger.info('Scraped 100 pages, Now exiting')
                    break
            except Exception as e:
                logger.error(f'Encounterd Error during the scraping of the {pages_scraped}th Page', exc_info=True)
    finally:
        driver.close()

: 

In [ ]:
scrape_pages(driver)

2025-09-27 18:05:58,591 [INFO] Started Scraping Pages
2025-09-27 18:05:58,592 [INFO] Trying to Scrape the 0th Page
2025-09-27 18:05:58,593 [INFO] Successfuly scraped the 0th Page
2025-09-27 18:05:58,594 [INFO] Trying to click on the 1th Page
2025-09-27 18:05:58,595 [INFO] Trying to go to the next Page
2025-09-27 18:06:00,739 [ERROR] Encounterd Error during the scraping of the 1th Page
Traceback (most recent call last):
  File "C:\Users\steve\AppData\Local\Temp\ipykernel_36672\1839121753.py", line 55, in scrape_pages
    go_to_next_page(driver)
  File "C:\Users\steve\AppData\Local\Temp\ipykernel_36672\1839121753.py", line 38, in go_to_next_page
    next_button.click()
  File "c:\Users\steve\web-scraping\.venv\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 119, in click
    self._execute(Command.CLICK_ELEMENT)
  File "c:\Users\steve\web-scraping\.venv\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 572, in _execute
    return self._parent.execute(comman

Next feature testing